### Install pakage and data exploratory analysis

In [ ]:
# import packages
import pyodbc
import pandas as pd
import sqlalchemy

In [ ]:
# connect to the database
cnxn = pyodbc.connect(driver = '{iSeries Access ODBC Driver}', system ='10.200.11.2', uid='YHSIEH',pwd = 'YHSIEH',LibraryList=['CONTAF','GUARDERIAF','REGALOF','PEDIDOSF','RETRIPLUSF','TRANSPORF','RTESF'])

In [ ]:
# create a cursor object
c1 = cnxn.cursor()

In [ ]:
# define sql command in order to get data from the specified table
# dataset1: Select data of clients that are currently active.
sql1 = "SELECT ESTADOCLIENTE, CODIGOCLIENTE, CIF, PAGRZ1, PAGRZ2, PROVINCIA, CODIGOGRUPO, GRUNOM AS NOMBREGRUPO, FECHA_ALTA, FECHA_BAJA FROM \
(SELECT * FROM ( \
SELECT P.PAGACT AS EstadoCliente,P.PAGCLI AS CodigoCliente,P.PAGRZ1,P.PAGRZ2,P.PAGPRF AS Provincia,P.PAGGRU AS CodigoGrupo, P.PAGFEC AS Fecha_Alta,P.PAGCIF AS CIF, B.PBJFEC AS Fecha_Baja FROM \
PEDIDOSF.PAGADOR AS P, PEDIDOSF.PAGBAJ AS B \
WHERE (P.PAGCLI = B.PBJCLI) AND (P.PAGFEC LIKE '1%' OR P.PAGFEC LIKE '2%') \
UNION ALL \
SELECT A.PAGACT AS EstadoCliente,A.PAGCLI AS CodigoCliente,A.PAGRZ1,A.PAGRZ2,A.PAGPRF AS Provincia,A.PAGGRU AS CodigoGrupo, A.PAGFEC AS Fecha_Alta,A.PAGCIF AS CIF, C.PBJFEC AS Fecha_Baja FROM \
GUARDERIAF.PAGADOR AS A, GUARDERIAF.PAGBAJ AS C \
WHERE (A.PAGCLI = C.PBJCLI) AND (A.PAGFEC LIKE '1%' OR A.PAGFEC LIKE '2%') \
UNION ALL \
SELECT D.PAGACT AS EstadoCliente,D.PAGCLI AS CodigoCliente,D.PAGRZ1,D.PAGRZ2,D.PAGPRF AS Provincia,D.PAGGRU AS CodigoGrupo, D.PAGFEC AS Fecha_Alta,D.PAGCIF AS CIF, E.PBJFEC AS Fecha_Baja FROM \
REGALOF.PAGADOR AS D, REGALOF.PAGBAJ AS E \
WHERE (D.PAGCLI = E.PBJCLI) AND (D.PAGFEC LIKE '1%' OR D.PAGFEC LIKE '2%') \
UNION ALL \
SELECT F.PAGACT AS EstadoCliente,F.PAGCLI AS CodigoCliente,F.PAGRZ1,F.PAGRZ2,F.PAGPRF AS Provincia,F.PAGGRU AS CodigoGrupo, F.PAGFEC AS Fecha_Alta,F.PAGCIF AS CIF, G.PBJFEC AS Fecha_Baja FROM \
RETRIPLUSF.PAGADOR AS F, RETRIPLUSF.PAGBAJ AS G \
WHERE (F.PAGCLI = G.PBJCLI) AND (F.PAGFEC LIKE '1%' OR F.PAGFEC LIKE '2%') \
UNION ALL \
SELECT H.PAGACT AS EstadoCliente,H.PAGCLI AS CodigoCliente,H.PAGRZ1,H.PAGRZ2,H.PAGPRF AS Provincia,H.PAGGRU AS CodigoGrupo, H.PAGFEC AS Fecha_Alta,H.PAGCIF AS CIF, I.PBJFEC AS Fecha_Baja FROM \
TRANSPORF.PAGADOR AS H, TRANSPORF.PAGBAJ AS I \
WHERE (H.PAGCLI = I.PBJCLI) AND (H.PAGFEC LIKE '1%' OR H.PAGFEC LIKE '2%')) AS J \
LEFT JOIN \
QGPL.CODGRUPF AS K \
ON J.CODIGOGRUPO = K.GRUCOD \
ORDER BY Fecha_Alta DESC)"

In [ ]:
df_client = pd.read_sql(sql1,cnxn)

In [ ]:
df_client.head()

,ESTADOCLIENTE,CODIGOCLIENTE,CIF,PAGRZ1,PAGRZ2,PROVINCIA,CODIGOGRUPO,NOMBREGRUPO,FECHA_ALTA,FECHA_BAJA
0,1,3215.0,A08718736,UNIQUE INTERIM,,BARCELONA,81.0,SESA START,20030116.0,20100712.0
1,0,8597.0,B71039002,INSTITUTO CIENTIFICO DE INNOV,Y TECNOLOGIAS APLICADAS,NAVARRA,99999.0,SIN GRUPO,20100128.0,0.0
2,0,8417.0,A80652928,"RANDSTAD EMPLEO ETT, S.A.",,MADRID,63.0,RANDSTAD,20090807.0,0.0
3,1,9872.0,B82690603,"JOSE CARGE, S.L.",,MADRID,99999.0,SIN GRUPO,20101011.0,20120521.0
4,0,30046.0,B17427709,"OIL ALBERA, S.L.",,GIRONA,99999.0,SIN GRUPO,20101013.0,0.0


In [ ]:
df_client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19683 entries, 0 to 19682
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ESTADOCLIENTE  19683 non-null  object 
 1   CODIGOCLIENTE  19683 non-null  float64
 2   CIF            19683 non-null  object 
 3   PAGRZ1         19683 non-null  object 
 4   PAGRZ2         19683 non-null  object 
 5   PROVINCIA      19683 non-null  object 
 6   CODIGOGRUPO    19683 non-null  float64
 7   NOMBREGRUPO    19680 non-null  object 
 8   FECHA_ALTA     19683 non-null  float64
 9   FECHA_BAJA     19683 non-null  float64
dtypes: float64(4), object(6)
memory usage: 1.5+ MB


In [ ]:
# Save this df as csv so I can access and work on the dataset.
df_client.to_csv("client.csv",sep=',')

In [ ]:
# define sql command in order to get data from the specified table
# dataset2: Select sales data (all-time) but only with active clients.
sql2 = "SELECT FACNUM, FACFEC, FACBRU, FACCOM, FACIVA, FACNET, FACCLI, FACTRA, 'Regalo' AS TIPO_PRODUCTO FROM REGALOF.FACCAB AS R \
WHERE FACFEC >= 20160000 \
UNION ALL \
SELECT FACNUM, FACFEC, FACBRU, FACCOM, FACIVA, FACNET, FACCLI, FACTRA, 'Guarderia' AS TIPO_PRODUCTO FROM GUARDERIAF.FACCAB AS G \
WHERE FACFEC >= 20160000 \
UNION ALL \
SELECT FACNUM, FACFEC, FACBRU, FACCOM, FACIVA, FACNET, FACCLI, FACTRA, 'ChequeGourmet' AS TIPO_PRODUCTO FROM CONTAF.FACCAB AS C \
WHERE FACFEC >= 20160000 \
UNION ALL \
SELECT FACNUM, FACFEC, FACBRU, FACCOM, FACIVA, FACNET, FACCLI, FACTRA, 'Retriplus' AS TIPO_PRODUCTO FROM RETRIPLUSF.FACCAB AS E \
WHERE FACFEC >= 20160000 \
UNION ALL \
SELECT FACNUM, FACFEC, FACBRU, FACCOM, FACIVA, FACNET, FACCLI, FACTRA, 'TarjetaTransporte' AS TIPO_PRODUCTO FROM TRANSPORF.TSFACCAB AS T \
WHERE FACFEC >= 20160000 \
UNION ALL \
SELECT FACNUM, FACFEC, FACBRU, FACCOM, FACIVA, FACNET, FACCLI, FACTRA, 'TarjetaComida' AS TIPO_PRODUCTO FROM PEDIDOSF.TSFACCAB AS A \
WHERE FACFEC >= 20160000 \
ORDER BY FACFEC DESC"

In [ ]:
df_sales = pd.read_sql(sql2,cnxn)

In [ ]:
df_sales.head()

,FACNUM,FACFEC,FACBRU,FACCOM,FACIVA,FACNET,FACCLI,FACTRA,TIPO_PRODUCTO
0,D020823,20210607.0,-639.0,0.00,0.00,-639.00,7439.0,0.0,ChequeGroumet
1,F000251,20210607.0,-2.0,0.00,-0.42,-2.42,1563.0,0.0,ChequeGroumet
2,T001655,20210607.0,-5.0,0.00,-1.05,-6.05,1563.0,0.0,ChequeGroumet
3,0104845,20210607.0,21743.0,0.00,0.00,21743.00,2382.0,0.0,ChequeGroumet
4,0104846,20210607.0,792.0,25.74,6.88,831.62,3475.0,5.0,ChequeGroumet


In [ ]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408687 entries, 0 to 408686
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FACNUM         408687 non-null  object 
 1   FACFEC         408687 non-null  float64
 2   FACBRU         408687 non-null  float64
 3   FACCOM         408687 non-null  float64
 4   FACIVA         408687 non-null  float64
 5   FACNET         408687 non-null  float64
 6   FACCLI         408687 non-null  float64
 7   FACTRA         408687 non-null  float64
 8   TIPO_PRODUCTO  408687 non-null  object 
dtypes: float64(7), object(2)
memory usage: 28.1+ MB


In [ ]:
# Save this df as csv so I can access and work on the dataset.
df_sales.to_csv("sales",sep=',')